# **Quantitative Momentum Strategy**

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

In [3]:
# Importing list of stocks 

stock_names = pd.read_csv('sp_500_stocks.csv')

In [ ]:
# Create a new dataFrame with specific information for the above stocks 
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
data_list = [] 

# Iterate through each stock ticker in the DataFrame
for ticker in stock_names['Ticker']:
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y") 

    try:
        last_price = hist.iloc[-1]['Close']
        year_ago_price = hist.iloc[0]['Close']
        one_year_return = (last_price - year_ago_price) / year_ago_price
    except IndexError: 
        last_price = None
        one_year_return = None

    new_row = {
        'Ticker': ticker,
        'Price': last_price,
        'One-Year Price Return': one_year_return,
        'Number of Shares to Buy': 0 
    }
    data_list.append(new_row) 

new_data = pd.DataFrame(data_list, columns=my_columns)

print(new_data)

In [5]:
final_dataframe = new_data
# Remove each stocks that is no longer available in the Exchange (delisted etc.)
final_dataframe = final_dataframe.dropna(subset=['Price'])

### **Removing Low-Momentum Sotcks**

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

### **Calculating the Numver of Shares to BUY**

In [8]:
final_dataframe = new_data

# Enter portofolio size in dollars
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(f"Portofolio size in euros : {portfolio_size}")
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    
# Remove each stocks that is no longer available in the Exchange
final_dataframe = final_dataframe.dropna(subset=['Price'])

# Calculate the number of shared for an equal weighted protofolio

final_dataframe.reset_index(drop=True, inplace=True)
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

Portofolio size in euros : 1e7


C:\Users\Altair\AppData\Local\Temp\ipykernel_15812\1588986627.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])


,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number Of Shares to Buy
0,A,149.759995,0.181634,0,143.0
1,AAL,14.400000,0.023454,0,1496.0
2,AAP,75.029999,-0.386156,0,287.0
3,AAPL,183.050003,0.060585,0,117.0
4,ABBV,160.750000,0.141109,0,134.0
...,...,...,...,...,...
459,YUM,137.619995,0.021301,0,156.0
460,ZBH,121.309998,-0.105656,0,177.0
461,ZBRA,315.799988,0.190440,0,68.0
462,ZION,44.320000,1.057462,0,486.0


### **Building a Better (and More Realistic) Momentum Strategy**

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

- High-quality momentum stocks show "slow and steady" outperformance over long periods of time
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns


In [35]:
hqm_columns = [ # hqm stands for high quility momentum
    'Ticker', 
    'Price', 
    'One-Year Price Return', 
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

data_list = []

# Iterate through each stock ticker
for ticker in stock_names['Ticker']:
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y")

    try:
        last_price = hist.iloc[-1]['Close']
        # Index directly for different periods
        year_ago_price = hist.iloc[0]['Close']
        six_month_ago_price = hist.iloc[max(0, len(hist) - 126)]['Close']  # Approx. 6 months
        three_month_ago_price = hist.iloc[max(0, len(hist) - 63)]['Close']  # Approx. 3 months
        one_month_ago_price = hist.iloc[max(0, len(hist) - 21)]['Close']  # Approx. 1 month

        one_year_return = (last_price - year_ago_price) / year_ago_price
        six_month_return = (last_price - six_month_ago_price) / six_month_ago_price
        three_month_return = (last_price - three_month_ago_price) / three_month_ago_price
        one_month_return = (last_price - one_month_ago_price) / one_month_ago_price
    except IndexError:  # In case the history data is shorter than expected
        last_price = one_year_return = six_month_return = three_month_return = one_month_return = None

    new_row = {
        'Ticker': ticker,
        'Price': last_price,
        'One-Year Price Return': one_year_return,
        'One-Year Return Percentile': 0,  # To be calculated later
        'Six-Month Price Return': six_month_return,
        'Six-Month Return Percentile': 0,  # To be calculated later
        'Three-Month Price Return': three_month_return,
        'Three-Month Return Percentile': 0,  # To be calculated later
        'One-Month Price Return': one_month_return,
        'One-Month Return Percentile': 0,  # To be calculated later
        'HQM Score': 0  # To be calculated later
    }
    data_list.append(new_row)

hqm_dataframe = pd.DataFrame(data_list, columns=hqm_columns)

A: No price data found, symbol may be delisted (period=1y)
ABC: No price data found, symbol may be delisted (period=1y)
ABMD: No price data found, symbol may be delisted (period=1y)
ALXN: No data found, symbol may be delisted
ANTM: No data found, symbol may be delisted
ATVI: No data found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (period=1y)
BLL: No data found, symbol may be delisted
BRK.B: No data found, symbol may be delisted
CERN: No data found, symbol may be delisted
COG: No data found, symbol may be delisted
CTL: No data found, symbol may be delisted
CTXS: No data found, symbol may be delisted
CXO: No data found, symbol may be delisted
DISCA: No data found, symbol may be delisted
DISCK: No data found, symbol may be delisted
DISH: No data found, symbol may be delisted
DRE: No data found, symbol may be delisted
ETFC: No data found, symbol may be delisted
FB: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted
FLIR: No dat

In [36]:
final_hqm_dataframe = hqm_dataframe
# Remove each stocks that is no longer available in the Exchange (delisted etc.)
final_hqm_dataframe = final_hqm_dataframe.dropna(subset=['Price'])
final_hqm_dataframe = final_hqm_dataframe.dropna(subset=['One-Year Price Return'])

### **Calculating Momentum Percentiles**

In [37]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in final_hqm_dataframe.index:
    for time_period in time_periods:
        final_hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(final_hqm_dataframe[f'{time_period} Price Return'], final_hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    # print(final_hqm_dataframe[f'{time_period} Return Percentile'])
    pass

C:\Users\Altair\AppData\Local\Temp\ipykernel_15812\2486798386.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.27292576419213976' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(final_hqm_dataframe[f'{time_period} Price Return'], final_hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
C:\Users\Altair\AppData\Local\Temp\ipykernel_15812\2486798386.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6004366812227074' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(final_hqm_dataframe[f'{time_period} Price Return'], final_hqm_dataframe.loc[row, f'{ti

### **Calculating the HQM Score**

We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

In [38]:
from statistics import mean

for row in final_hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(final_hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    final_hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

C:\Users\Altair\AppData\Local\Temp\ipykernel_15812\3145735095.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.49672489082969434' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


### **Selecting the 50 Best Momentum Stocks**

In [39]:
final_hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
final_hqm_dataframe = final_hqm_dataframe[:51]

### **Calaculating the Number of Shares to BUY**

In [40]:
# Enter portofolio size in dollars
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(f"Portofolio size in euros : {portfolio_size}")
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    
# Remove each stocks that is no longer available in the Exchange
final_hqm_dataframe = final_hqm_dataframe.dropna(subset=['Price'])

# Calculate the number of shared for an equal weighted protofolio

final_hqm_dataframe.reset_index(drop=True, inplace=True)
position_size = float(portfolio_size) / len(final_hqm_dataframe.index)
for i in range(0, len(final_hqm_dataframe['Ticker'])):
    final_hqm_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_hqm_dataframe['Price'][i])
final_hqm_dataframe

Portofolio size in euros : 1e7


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number Of Shares to Buy
0,AAL,14.400000,0,0.023454,0.272926,0.225532,0.600437,-0.035499,0.251092,0.095057,0.862445,0.496725,13616.0
1,AAP,75.029999,0,-0.386156,0.010917,0.346300,0.801310,0.139613,0.794760,0.064255,0.711790,0.579694,2613.0
2,AAPL,183.050003,0,0.060585,0.325328,0.007477,0.126638,-0.020581,0.299127,0.038223,0.576419,0.331878,1071.0
3,ABBV,160.750000,0,0.141109,0.458515,0.186623,0.515284,-0.062511,0.189956,-0.009428,0.299127,0.365721,1219.0
4,ABT,104.739998,0,-0.028955,0.211790,0.130603,0.358079,-0.064610,0.185590,-0.040051,0.174672,0.232533,1872.0
5,ACN,306.329987,0,0.142410,0.460699,-0.017761,0.096070,-0.163062,0.050218,-0.028757,0.220524,0.206878,640.0
6,ADBE,482.290009,0,0.411939,0.770742,-0.165213,0.019651,-0.211738,0.024017,0.017296,0.451965,0.316594,406.0
7,ADI,207.190002,0,0.169211,0.500000,0.251404,0.650655,0.072746,0.574236,0.077599,0.783843,0.627183,946.0
8,ADM,62.980000,0,-0.136273,0.096070,-0.113502,0.034934,0.171939,0.884279,0.032967,0.548035,0.390830,3113.0
9,ADP,246.860001,0,0.203465,0.554585,0.102733,0.277293,-0.007134,0.331878,0.012344,0.423581,0.396834,794.0


### **Formating the result in excel**

Using XlsxWriter

In [49]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_hqm_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template],
                    'M': ['Number of Shares to Buy', integer_template]
                    }

writer.close()